In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents
extracted_data = load_pdf("data/")
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings
embeddings = download_hugging_face_embeddings()
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key='pcsk_2kFykp_KhvdnsyhiF5oM2RS9iJqiyFvQNznDBkqw12TocYxz7VzdT82aiWqAt9rud11M7H')
index = pc.create_index(
    name='quickstart',
    dimension=384,
    metric='cosine',
    deletion_protection='enabled',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(documents)

embedding_dim = embeddings.shape[1]

print(f"Embedding Dimensions: {embedding_dim}")
from langchain_pinecone import Pinecone

vectorstore = Pinecone(index=index, embedding=embedding_model)

import math

# Function to split vectors into smaller batches
def batch_vectors(vectors, batch_size):
    for i in range(0, len(vectors), batch_size):
        yield vectors[i : i + batch_size]

# Example: Split data into batches of size 100
batch_size = 100  # Adjust this size based on your payload
for batch in batch_vectors(vectors, batch_size):
    upsert_response = index.upsert(
        vectors=batch,
        namespace="example-namespace"
    )
print("All vectors upserted successfully!")

# Ensure each chunk is a string and create Document objects
documents = [Document(page_content=str(chunk)) for chunk in text_chunks]

# Extract the text content from the documents (list of strings)
text_content = [doc.page_content for doc in documents]

# Embed the documents using the Hugging Face model
embedded_vectors = embedding_model.embed_documents(text_content)

# Function to split data into smaller batches
def batch_upsert(vectors, batch_size=100):
    # Split the vectors into smaller batches
    for i in range(0, len(vectors), batch_size):
        yield vectors[i:i + batch_size]

# Prepare the upsert data in the required format
pinecone_data = [
    (
        str(i),  # Unique ID for each vector (can be an incremental ID or anything unique)
        embedded_vectors[i],  # The actual vector for the chunk
        {"text": text_content[i]}  # Metadata: the original chunk as the 'text' field
    )
    for i in range(len(text_chunks))
]

# Split the upsert data into batches
batches = batch_upsert(pinecone_data, batch_size=100)  # Adjust the batch_size as needed

# Upsert the vectors in batches
for batch in batches:
    upsert_response = index.upsert(
        vectors=batch,
        namespace="example-namespace"  # Replace with your desired namespace
    )

print("Upsert completed successfully.")
query_vector = embedding_model.embed_query("How are your allergies") # Replace with your own query vector

# Query the Pinecone index to find the top 5 most similar vectors
query_response = index.query(
    vector=query_vector,  # The query vector
    top_k=5,              # Number of similar vectors to retrieve
    include_metadata=True,  # Include metadata in the response (optional)
    namespace="example-namespace"  # Namespace you used during upsert
)

# Display the query results
"""for match in query_response['matches']:
    print(f"{match['metadata']}")"""
print(query_response)
prompt_template="""Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:"""

PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}
llm=CTransformers(model="D:\pinecone\model\llama-2-13b-chat.ggmlv3.q5_1.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vectorstore.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)
user_input=input(f"Input Prompt:")    
result=qa({"query": user_input}) print("Response : ", result["result"])


AttributeError: 'list' object has no attribute 'shape'